<a href="https://colab.research.google.com/github/kimhalyn/OPEN-API/blob/main/mntwal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 라이브러리
# colab에 xmltodict 라이브러리가 기본 설치되지 않으므로 별도 설치(외부 라이브러리)
!pip install xmltodict 
from urllib.request import urlopen, Request
from urllib.parse import urlencode, unquote, quote_plus
from bs4 import BeautifulSoup
import pandas as pd
import requests, xmltodict, json
from xml.etree.ElementTree import Element, SubElement, ElementTree
import xml.etree.ElementTree as ET
import pickle
import numpy as np

In [ ]:
\from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from time import sleep
import csv

url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/mntwal/list' # !!확인
key = 'tMF2MGGUmsNNPf15Nr91+UA4YVQiWXVjLS7R0odxGSR0yfNjcHFkdRpCT6hXbILlBP1T+TnDJYz1kkDjiaIiQg==' # !!확인
 
api_key = requests.utils.unquote(key) #디코딩

# 댐코드 및 수위 관측소 코드 불러오기
data1 = pd.read_csv('/content/drive/MyDrive/water_resource_csv/walcode.csv') 
data2 = pd.read_csv('/content/drive/MyDrive/water_resource_csv/walcode.csv') 

# 댐코드 및 수위 관측소 코드 조회(!!열 지정 주의하기)
damcode = data1.iloc[0:5, 1] 
walcode = data2.iloc[0:5, 3]
xmlrows = []

for code1 in damcode:
  for code2 in walcode:
    
    Params = '?' + urlencode({quote_plus('ServiceKey') : api_key,
                                  quote_plus('sdate') : "2020-01-01",
                                  quote_plus('stime') : "00",
                                  quote_plus('edate') : "2020-01-10",
                                  quote_plus('etime') : "24",
                                  quote_plus('damcode') : code1, #댐코드
                                  quote_plus('wal') : code2, #수위 관측소 코드
                                  quote_plus('tms') : "10", #10분 단위
                                  quote_plus('numOfRows') : "80000",
                                  quote_plus('pageNo') : "1"})
    
#  print(Params)
  
  res = requests.get(url + Params).text.encode('utf-8') # 요청에 대한 응답내용(.encode빼기도 함)
  sleep(0.5)
  print(res) 
  
  xmlobj = bs4.BeautifulSoup(res, 'lxml') #lxml 파서(구문분석)  'lxml-xml'
  print(xmlobj)
  
  #item 태그 분리
  rows = xmlobj.findAll('item') # 매치되는 문자열을 리스트로 리턴
  xmlrows.append(rows)

print (xmlrows)


b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><flux>0</flux><no>1440</no><obsrdtmnt>2020-01-01 00:10</obsrdtmnt><wal>0.04</wal></item><item><flux>0</flux><no>1439</no><obsrdtmnt>2020-01-01 00:20</obsrdtmnt><wal>0.04</wal></item><item><flux>0</flux><no>1438</no><obsrdtmnt>2020-01-01 00:30</obsrdtmnt><wal>0.04</wal></item><item><flux>0</flux><no>1437</no><obsrdtmnt>2020-01-01 00:40</obsrdtmnt><wal>0.04</wal></item><item><flux>0</flux><no>1436</no><obsrdtmnt>2020-01-01 00:50</obsrdtmnt><wal>0.04</wal></item><item><flux>0</flux><no>1435</no><obsrdtmnt>2020-01-01 01:00</obsrdtmnt><wal>0.04</wal></item><item><flux>0</flux><no>1434</no><obsrdtmnt>2020-01-01 01:10</obsrdtmnt><wal>0.04</wal></item><item><flux>0</flux><no>1433</no><obsrdtmnt>2020-01-01 01:20</obsrdtmnt><wal>0.04</wal></item><item><flux>0</flux><no>1432</no><obsrdtmnt>2020-01-01 01:30</obsrdtmnt><wal>0.04</wal

In [26]:
print(Params)

?ServiceKey=jkd4PnVjssNP%2Bg2WeRzs8lgc4SOelLsncvuMr81%2F9R9nSYKkiQAcb2xt8GTYgTJSQGQ4%2BhtuRuyIXSwRXr210w%3D%3D&sdate=2020-01-01&stime=00&edate=2020-01-02&etime=24&damcode=1001210&wal=9000268&tms=10&numOfRows=80000&pageNo=1


In [22]:
print(damcode)

0    1007601
1    1007601
2    1001210
3    1001210
4    1001210
Name: Damcode, dtype: int64


In [23]:
print(walcode)

0    1007601
1    1005681
2    9000266
3    9000264
4    9000268
Name: walobsrvtcode, dtype: int64


In [ ]:

filePath = '/content/drive/MyDrive/water_resource_csv/xmlrows.xml'


with open(filePath, 'wb') as lf:
    pickle.dump(xmlrows, lf)


with open(filePath, 'rb') as lf:
    readList = pickle.load(lf)


In [43]:
# 컬럼 생성
rowList = []
nameList = []
columnList = []
xmlrowsLen = len(xmlrows)
target = 0
for i in range(0, xmlrowsLen):
    
   
    if len(xmlrows[i]) == 0:
       target = target + 1
      
    else:
        break 
columns = xmlrows[target][0].find_all()
columnsLen = len(columns)
for i in range(0, columnsLen):
    nameList.append(columns[i].name)
nameList = ["Damcode"] + ["walobsrvtcode"] + nameList

In [ ]:
xmlrows[0]

In [29]:
print(nameList)

['Damcode', 'walobsrvtcode', 'flux', 'no', 'obsrdtmnt', 'wal']


In [44]:
total = []
damcodeLen = len(damcode)
walcodeLen = len(walcode)


def el_code(a): 
    columnList = []
    eachrows = xmlrows[a]
    eachrowslen = len(eachrows)
    for i in range(0, eachrowslen):
        columns = eachrows[i].find_all()
        columnsLen = len(columns)
        for j in range(0, columnsLen):
            eachColumn = columns[j].text 
            columnList.append(eachColumn)
        columnList = [damcode[a]] + [walcode[a]] + columnList
        rowList.append(columnList)
        columnList = []
    return rowList



for a in range(0, damcodeLen, walcodeLen):
    rowList=el_code(a)
    total.append(rowList)

result = pd.DataFrame(total[0], columns=nameList)

   

In [45]:
print(result)

      Damcode  walobsrvtcode flux    no         obsrdtmnt   wal
0     1007601        1007601    0  1440  2020-01-01 00:10  0.04
1     1007601        1007601    0  1439  2020-01-01 00:20  0.04
2     1007601        1007601    0  1438  2020-01-01 00:30  0.04
3     1007601        1007601    0  1437  2020-01-01 00:40  0.04
4     1007601        1007601    0  1436  2020-01-01 00:50  0.04
...       ...            ...  ...   ...               ...   ...
1435  1007601        1007601    0     5  2020-01-10 23:20  0.21
1436  1007601        1007601    0     4  2020-01-10 23:30  0.21
1437  1007601        1007601    0     3  2020-01-10 23:40  0.21
1438  1007601        1007601    0     2  2020-01-10 23:50  0.21
1439  1007601        1007601    0     1  2020-01-10 24:00  0.21

[1440 rows x 6 columns]


In [46]:
result.to_csv("/content/drive/MyDrive/water_resource_csv/mntwal_test2.csv",encoding="utf-8-sig")

In [ ]:
result